In [1]:
from __future__ import print_function
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle 
import os.path 
import base64 
import email 
from bs4 import BeautifulSoup 
import os
import dateutil.parser as parser
import csv
import re
from datetime import date, timedelta

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is created automatically when the authorization 
    # flow completes for the first time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    #get labels
    label_results = service.users().labels().list(userId='me').execute()
    labels = label_results.get('labels', [])
    # print(labels)
    
    today = date.today()
    yesterday = today - timedelta(365)
    query = "from: {0} before: {1} after: {2}".format('info@mailer.netflix.com', today.strftime('%Y/%m/%d'), yesterday.strftime('%Y/%m/%d'))

    #get messages
    label_id = 'INBOX'
    message_results = service.users().messages().list(userId='me', maxResults=500, labelIds=[label_id], q= query).execute()
    mssg_list = message_results['messages']
    
    temp_dict = {}
    temp_list = [] 
    final_list = []
    for mssg in mssg_list:
        m_id = mssg['id'] # get id of individual message
        message = service.users().messages().get(userId='me', id=m_id).execute() # fetch the message using API
        payld = message['payload'] # get payload of the message 
        headr = payld['headers'] # get header of the payload

        for one in headr: # getting the Sender
            if one['name'] == 'From':
                msg_from = one['value']
                matches = re.findall('<\S+?>', msg_from)  
                filtered = [ x[1:-1] for x in matches if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", x[1:-1]) ]
                sender = filtered[0] if len(filtered) > 0 else None
                temp_dict['Sender'] = sender
            else:
                pass
               
        for two in headr: # getting the date
            if two['name'] == 'Date':
                msg_date = two['value']
                date_parse = (parser.parse(msg_date))
                m_date = (date_parse.date())
                temp_dict['Date'] = str(m_date)
            else:
                pass
            
        for three in headr: # getting the Subject
            if three['name'] == 'Subject':
                msg_subject = three['value']
                temp_dict['Subject'] = msg_subject
            else:
                pass

        temp_dict['Snippet'] = message['snippet'].replace("\u200c", "").rstrip() #getting the Snippet
        temp_list = list(temp_dict.values())
        final_list.append(temp_list)
    print(final_list)

    with open('x.csv', 'w', encoding='utf-8', newline = '') as csvfile:
        write = csv.writer(csvfile) 
        write.writerow(['Sender','Date','Subject','Snippet'])
        write.writerows(final_list)
        
if __name__ == '__main__':
    main()

[['info@mailer.netflix.com', '2020-11-17', 'Gayatri, it’s been 78 days. Fancy watching something?', 'For ₹199 a month, put your feet up and relax with the titles you love.'], ['info@mailer.netflix.com', '2020-11-12', 'Gayatri, we have a plan for every budget – as low as ₹199 a month', 'Watch the films and programmes you love. Start today.'], ['info@mailer.netflix.com', '2020-11-10', 'Gayatri, come back for just ₹199', 'Hi Gayatri When you watch Netflix, your viewing preferences are saved so we can recommend more of the TV programmes and films you know and love. Come back today to: • Pick up where you left off with'], ['info@mailer.netflix.com', '2020-11-07', 'Gayatri, here are 3 great reasons to come back to Netflix.', 'Hi Gayatri Here are 3 great reasons to come back to Netflix: • Discover brand new TV programmes &amp; films everyone&#39;s talking about. We&#39;ll suggest new titles tailored to your taste. • Rewatch'], ['info@mailer.netflix.com', '2020-11-05', 'Try Netflix again today

In [3]:
#find out what keywords indicate the user is a subscriber or not